In [16]:
!pip install praw pandas

This dataset gathered data from Reddit using praw (The Python Reddit API Wrapper).
The API connection process is conducted under the guidence from the link below:

https://praw.readthedocs.io/en/latest/getting_started/quick_start.html


Your username is: yyd859
Your password is: 54804839
Your app's client ID is: XRBFUnVyp6VVdknjdodIKQ(不确定是不是这个)
Your app's client secret is: rPqYL1OJMiKoek-7VGWOHoQvFbUfpQ

In [75]:
import praw

reddit = praw.Reddit(
    client_id="XRBFUnVyp6VVdknjdodIKQ",
    client_secret="rPqYL1OJMiKoek-7VGWOHoQvFbUfpQ",
    user_agent="Mac:yyd859.myredditapp:v0.1 (by /u/yyd859)",
)
#Connection Check
print(reddit.read_only)

True


In [114]:
#######
# IMPORT PACKAGES
#######

import praw
import pandas as pd


# Acessing the reddit api


reddit = praw.Reddit(client_id="XRBFUnVyp6VVdknjdodIKQ",#my client id
                     client_secret="rPqYL1OJMiKoek-7VGWOHoQvFbUfpQ",  #your client secret
                     user_agent="Mac:yyd859.myredditapp:v0.1 (by /u/yyd859)", #user agent name
                     username = "yyd859",     # your reddit username
                     password = "54804839")     # your reddit password

sub = ['wallstreetbets',
'investing',
'personalfinance',
'stocks',
'stockmarket',
'superstonk',
'antiwork',
'mademesmile',
'IPO',
'AskReddit',
'Wallstreetbetsnew',
'CryptoCurrency',
'conspiracy',
'UrvinFinance',
'Infinity_For_Reddit',
'OutOfTheLoop',
'collapse',
'GME',
'mauerstrassenwetten',
'unclebens',
'HailCorporate',
'Bogleheads',
'preppers']  # make a list of subreddits you want to scrape the data from


for s in sub:
    subreddit = reddit.subreddit(s)   # Chosing the subreddit


########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword
    query = ['reddit IPO']
#search by relevance
    for item in query:
        post_dict = {
            "title" : [],
            "score" : [],
            "id" : [],
            "url" : [],
            "comms_num": [],
            "created" : [],
            "body" : []
        }
        comments_dict = {
            "comment_id" : [],
            "comment_parent_id" : [],
            "comment_body" : [],
            "comment_link_id" : []
        }
        for submission in subreddit.search(query,sort='relevance',time_filter="month",limit = 20):
            post_dict["title"].append(submission.title)
            post_dict["score"].append(submission.score)
            post_dict["id"].append(submission.id)
            post_dict["url"].append(submission.url)
            post_dict["comms_num"].append(submission.num_comments)
            post_dict["created"].append(submission.created)
            post_dict["body"].append(submission.selftext)
            
            ##### Acessing comments on the post
            submission.comments.replace_more(limit = 1)
            for comment in submission.comments.list():
                comments_dict["comment_id"].append(comment.id)
                comments_dict["comment_parent_id"].append(comment.parent_id)
                comments_dict["comment_body"].append(comment.body)
                comments_dict["comment_link_id"].append(comment.link_id)
        
        post_comments = pd.DataFrame(comments_dict)

        post_comments.to_csv(s+"_comments_relevance_"+ item +"_subreddit.csv")
        post_data = pd.DataFrame(post_dict)
        post_data.to_csv(s+"_posts_relevance"+ item +"_subreddit.csv")

In [115]:
for s in sub:
    subreddit = reddit.subreddit(s)
    query = ['reddit IPO']
#search by top
    for item in query:
        post_dict = {
            "title" : [],
            "score" : [],
            "id" : [],
            "url" : [],
            "comms_num": [],
            "created" : [],
            "body" : []
        }
        comments_dict = {
            "comment_id" : [],
            "comment_parent_id" : [],
            "comment_body" : [],
            "comment_link_id" : []
        }
        for submission in subreddit.search(query,sort='top',time_filter="month",limit = 20):
            post_dict["title"].append(submission.title)
            post_dict["score"].append(submission.score)
            post_dict["id"].append(submission.id)
            post_dict["url"].append(submission.url)
            post_dict["comms_num"].append(submission.num_comments)
            post_dict["created"].append(submission.created)
            post_dict["body"].append(submission.selftext)
            
            ##### Acessing comments on the post
            submission.comments.replace_more(limit = 1)
            for comment in submission.comments.list():
                comments_dict["comment_id"].append(comment.id)
                comments_dict["comment_parent_id"].append(comment.parent_id)
                comments_dict["comment_body"].append(comment.body)
                comments_dict["comment_link_id"].append(comment.link_id)
        
        post_comments = pd.DataFrame(comments_dict)

        post_comments.to_csv(s+"_comments_top_"+ item +"_subreddit.csv")
        post_data = pd.DataFrame(post_dict)
        post_data.to_csv(s+"_posts_top"+ item +"_subreddit.csv")

In [116]:
#simply merge
import os, glob
import pandas as pd


path = "/Users/yyd/Documents/Duke Fuqua MQM/Jupyter Notebook/Formula1-RedditForum-SentimentAnalysis"


comments_files = glob.glob(os.path.join(path, "*_comments_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in comments_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "comments.csv")

comments_files = glob.glob(os.path.join(path, "*_posts_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in comments_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "posts.csv")


In [ ]:
#remove duplicate